In [1]:
import spacy
import pandas as pd
import numpy as np
import neuralcoref

data_raw = pd.read_csv('pictures_sample.csv')
data_english = data_raw[data_raw.Language == 'EN']
def author_del(text):
    text_s = text
    while(text_s[-1] != '\n' and len(text_s) > 1): 
        text_s = text_s[:-1]
    if(len(text_s) <= 1):
        return(text)
    return(text_s[:-1])
data_english.Description = data_english.Description.apply(author_del)          
data_english.drop_duplicates(subset='Description', keep='first', inplace=True)
ind = list(data_english.index)

print(data_english.shape)


C:\Users\THINKPAD\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: spacy.morphology.Morphology size changed, may indicate binary incompatibility. Expected 104 from C header, got 112 from PyObject
  return f(*args, **kwds)
C:\Users\THINKPAD\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: spacy.vocab.Vocab size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
  return f(*args, **kwds)
C:\Users\THINKPAD\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (3,21,22,33,35,39,43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\THINKPAD\Anaconda3\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pyd

(82608, 54)


In [2]:
#On importe le fichier contenant les scores de chaque fonction grammaticale

scoring = pd.read_csv('scoring.csv', delimiter = ";")
dict_val = {}

for i in range(48):
    dict_val[scoring['function'][i]] = scoring['score_norm'][i]

In [3]:
nlp=spacy.load("en_core_web_md")  #on importe un modèle md pour avoir des mots vectorisés
neuralcoref.add_to_pipe(nlp,greedyness=0.5)

In [37]:
def dep_ent(ent, doc):
    """ Retourne la fonction grammaticale :  la 'dep', d'une entité. Cette fonction est nécessaire car elle permet d'affecter
    une dep à une entité composée de plusieurs mots ayant chacun une dep de base.
    Traite aussi le cas particulier des mots étant des conj ou des compound : leur vrai dep et celle du mot auxquels
    ils sont associés en tant que conj ou compound."""
    start= ent.start
    end=ent.end
    for k in range(start,end):
        if doc[k].head.text not in ent.text: 
            if doc[k].dep_=='conj':     
                tok=doc[k]            
                while tok.dep_=='conj':
                    tok=tok.head      
                return(tok.dep_)
            
            if doc[k].dep_=='compound':   
                tok=doc[k]            
                while tok.dep_=='compound':
                    tok=tok.head      
                return(tok.dep_)
            return(doc[k].dep_)    
    return doc[start].dep_

def ent_good_type(ent): #filtre les entités selon leur type
    return (ent.label_ == "PERSON"or ent.label_ == "NORP" or ent.label_ == "ORG" or ent.label_ == "GPE" or ent.label_ == "EVENT" or ent.label_ == "LOC")

def sort_ent(doc):
    """Retourne la liste des entités en les filtrant selon leur type et en les triant de manière à avoir au début de
    la liste les entités ayant des coréférences."""
    ent_coref=[ent for ent in doc.ents if ent._.is_coref and ent_good_type(ent)]
    ent_vanilla=[ent for ent in doc.ents if ent_good_type(ent) and not ent._.is_coref]
    return ent_coref + ent_vanilla

def scores_doc(doc):
    """Retourne le score de chaque entité pour la méthode sans neuralcoref."""
    res={}
    for ent in doc.ents:
        if ent.text not in res.keys():
            res[ent.text]=dict_val[dep_ent(ent,doc)]
        else:
            res[ent.text]+=dict_val[dep_ent(ent,doc)]
    return res

def scores_doc_coref1(doc):
    """Retourne le score de chaque entité pour la méthode de base opérée sur le document resolved.
    Le document resolved est le document de base dans lequel toutes les références à un groupe de mot sont remplacées
    par celui-ci.
    Par exemple : My dad is home. He watches TV devient My dad is home. My dad watches TV.
    En raisonnant avec le nlp sur le document resolved, le nlp va détecter beaucoup plus de fois la même entité.
    Le principal inconvénient est que toutes les références sont remplacées, y compris celles qui ne sont pas associées 
    à des entités mais plutôt à des très longs bouts de phrase qui sont repris par un pronom comme "it"  """
    
    doc=nlp(doc._.coref_resolved)
    res={}
    for ent in doc.ents:
        if ent.text not in res.keys():
            res[ent.text]=dict_val[dep_ent(ent,doc)]
        else:
            res[ent.text]+=dict_val[dep_ent(ent,doc)]
    return res

def is_in_cluster(ent,cluster):  #détermine si une entité est dans une des mentions d'un cluster
    for span in cluster.mentions:
        if ent.text in span.text:
            return True
    return False



def scores_doc_coref2(doc):
    """Cette méthode utilise neuralcoref mais au lieu d'agir sur le doc resolved, on va chercher a l'interieur des clusters
    associés au document. Pour rappel, un cluster contient un mot et ses références. Ce mot n'est pas forcément une entité.
    Un cluster est de la forme Trump : [Trump, he].
    Pour chaque entité, on va d'abord regarder si elle est coréférencée (donc mentionnée explicitement dans un cluster)
    ou pas.
    Si elle est coréférencée, on va chercher le cluster qui lui est associé.
    Une fois dans ce cluster, on va regarder pour chaque mention (réf à l'entité) si elle contient ou pas des entités.
    Si c'est le cas, on va alors indiquer que cette entité ne doit pas être retraitée par la suite car elle est traitée en
    tant qu'autre mention de l'entité que l'on traite actuellement.
    Par exemple, pour l'entité référencée Trump, on trouve le cluster suivant : Trump : [Trump, Donald Trump, he].
    Alors l'entité Donald Trump va être marquée comme traitée car elle est en fait équivalente à Trump.
    "he" n'étant pas une entité on a pas ce pb pour cette mention la.
    Ensuite on calcule le score de chaque mention (Trump, Donald Trump et he) en se basant sur leur fonction grammaticale.
    Puis on ajoute ces scores à celui de l'entité de base (Trump).
    
    Si l'entité de base n'est pas référencée, c'est un peu le même principe, la seule complexité en plus étant le fait que
    l'entité n'est pas forcément mentionnée telle quelle dans les mentions d'un cluster. 
    Par exemple, si notre entité de base est Sienna Miller, il est possible qu'on ait un cluster de la forme
    The british actress Sienna Miller : [The british actress Sienna Miller, she].
    Dans ce cas il faut rechercher si chaque mention contient l'entité de base avant de procéder comme pour les 
    entités référencées.
    
    """
    clusters=doc._.coref_clusters  #liste des clusters du doc. 
    res={} #le resultat sera un dictionnaire qui permet d'associer un score à chque entité
    ent_treated={} #dictionnaire pour différencier les entités traitées des autres
    ents_sorted=sort_ent(doc)  #on place les entités ayant une coref avant les autres
    for ent in ents_sorted:   
        ent_treated[ent.text]=0  #on initialise en affectant 0 à toutes les entités

    for ent in ents_sorted:     #pour chaque entité
        if ent_treated[ent.text]==0: #si elle n'est pas considérée comme "traitée"
            if ent._.is_coref:  #si elle est coréférencée (donc si elle apparait en tant que mention pour un cluster )
                #print(ent)
                for cluster in clusters:  #on va chercher le cluster associé à l'entité puisqu'elle est coref
                    if ent in cluster.mentions: # on regarde si l'entité est dans les mentions du cluster
                        for span in cluster.mentions: #Mtnt qu'on est dans le bon cluster, on regarde pour chaque span
                            if span.ents != [] : 
                                max_sim=ent.similarity(span.ents[0])
                                max_span_ent=span.ents[0]
                                for span_ent in span.ents: #on cherche l'entité dans le span la plus proche de ent
                                    if ent.similarity(span_ent) > max_sim:
                                        max_span_ent=span_ent
                                        max_sim=ent.similarity(span_ent)
                                ent_treated[max_span_ent.text]=1 #une fois qu'on l'a trouvé on la marque comme traitée
                            else:
                                max_span_ent=span        
                            if ent.text not in res.keys(): 
                                res[ent.text]=dict_val[dep_ent(max_span_ent,doc)]  #on affecte le score de l'entité
                            else:
                                res[ent.text] += dict_val[dep_ent(max_span_ent,doc)] 
            else: #si l'entité n'est pas exactement coréférencée par neuralcoref
                flag=0
                for cluster in clusters:
                    if is_in_cluster(ent,cluster) and ent.label_ != 'NORP': 
                        flag=1
                        for span in cluster.mentions:
                            #print(span)
                            if span.ents != [] :
                                max_sim=ent.similarity(span.ents[0])
                                max_span_ent=span.ents[0]
                                for span_ent in span.ents:
                                    if ent.similarity(span_ent) > max_sim:
                                        max_span_ent=span_ent
                                        max_sim=ent.similarity(span_ent)
                                ent_treated[max_span_ent.text]=1
                            else:
                                max_span_ent=span #on peut peut etre supprimer cette branche else et...
                            if ent.text not in res.keys():
                                res[ent.text]=dict_val[dep_ent(max_span_ent,doc)] #...replacer max_span_ent par span ici
                            else:
                                res[ent.text] += dict_val[dep_ent(max_span_ent,doc)]  #et ici
                if flag==0: #si l'entité n'est vraiment dans aucun cluster   
                    if ent.text not in res.keys():
                        res[ent.text]=dict_val[dep_ent(ent,doc)]
                    else:
                        res[ent.text]+=dict_val[dep_ent(ent,doc)] 
    #print(ent_treated)
    return res
    
doc=nlp('Today, Donald Trump met Macron in France. Trump wants this country to be happy.')
#doc=nlp(data_english.Description[4])
dep_ent(doc.ents[0],doc)
print(doc)
print(doc.ents)
print(scores_doc_coref2(doc))
print(scores_doc_coref1(doc))
print(scores_doc(doc))


Today, Donald Trump met Macron in France. Trump wants this country to be happy.
(Today, Donald Trump, Macron, France, Trump)
{'Donald Trump': 2.0, 'France': 1.829787234, 'Macron': 0.872340426}
{'Today': 0.70212766, 'Donald Trump': 2.0, 'Macron': 0.872340426, 'France': 1.829787234}
{'Today': 0.70212766, 'Donald Trump': 1.0, 'Macron': 0.872340426, 'France': 0.8297872340000001, 'Trump': 1.0}


In [5]:


#doc=nlp("The owners of a Tucson-area Mexican restaurant are fending off social media attacks after appearing in the VIP area at a Trump rally in Phoenix last month. The Arizona Daily Star reported that a Facebook group posted a screenshot image from the rally that showed Sammy’s Mexican Grill co-owner Betty Rivas standing behind Trump, wearing a red cowboy hat emblazoned with Latinos Love Trump. The newspaper says the post attracted more than 230 comments, almost all negative. Sammy’s co-owner Jorge Rivas said some people also posted “very ugly stuff” on social media including the restaurant’s Yelp and Google pages. On Sunday Trump tweeted support for the Rivas’ restaurant. Despite apparently watching a segment on the eatery on his favourite Fox News magazine show, Fox & Friends, the president got its location wrong. The food is GREAT at Sammy’s Mexican Grill in Phoenix, Arizona, the tweet said. Congratulations to Betty & Jorge Rivas on doing such a wonderful job. I will try hard to stop by the next time I am in Phoenix. Phoenix and Tucson are 113 miles apart. Trump’s hardline immigration policies make him unpopular with most Latino Americans. Rivas said he and his wife posted a video on Facebook defending their rights as naturalized American citizens to vote, support and meet whomever they please. Just because we are Latinos it doesn’t mean that we have to feel like every other Latino in this country, Rivas said. We are individuals and we feel that we have the constitutional right to meet and support whoever we want. Jorge Rivas said the online attacks had little to no impact on business at the restaurant which he and his wife opened in 1996.")

doc=nlp(data_english.Description[0])
print(doc)
print('\n')
print(doc.ents)
print('\n')
for X in doc.ents:
    print(X,dict_val[dep_ent(X,doc)],dep_ent(X,doc))
print('\n')

print(doc._.coref_clusters)
print('\n')

L=scores_doc_coref2(doc)
print("Résultat méthode 2 : ", dict(sorted(L.items(), key=lambda t: t[1])))
print('\n')

L=scores_doc_coref1(doc)
print("Résultat méthode 1 : ", dict(sorted(L.items(), key=lambda t: t[1])))
print('\n')

L=scores_doc(doc)
print("Résultat sans neuralcoref : ", dict(sorted(L.items(), key=lambda t: t[1])))


US-British actress Sienna Miller poses after unveiling her dedicated beach locker room on the Promenade des Planches at the 45th Deauville US Film Festival in Deauville, western France, on September 11, 2019. 


(US, British, Sienna Miller, the Promenade des Planches, 45th, Deauville US Film Festival, Deauville, France, September 11, 2019)


US 0.489361702 amod
British 0.489361702 amod
Sienna Miller 1.0 nsubj
the Promenade des Planches 0.8297872340000001 pobj
45th 0.489361702 amod
Deauville US Film Festival 0.8297872340000001 pobj
Deauville 0.8297872340000001 pobj
France 0.46808510600000003 appos
September 11, 2019 0.8297872340000001 pobj


[US-British actress Sienna Miller: [US-British actress Sienna Miller, her]]


Résultat méthode 2 :  {'France': 0.46808510600000003, 'British': 0.489361702, 'Deauville US Film Festival': 0.8297872340000001, 'Deauville': 0.8297872340000001, 'US': 0.872340425, 'Sienna Miller': 1.382978723}


Résultat méthode 1 :  {'France': 0.46808510600000003, '45th':

In [11]:
import time

start_time=time.time()

nlp_list = []
for desc in data_english.Description[:10000]:
    nlp_list.append(nlp(desc))

print("Temps d'exécution : ", time.time()-start_time, "s")

Temps d'exécution :  832.3939981460571 s


In [35]:
nlp_list

AttributeError: 'list' object has no attribute 'head'

In [44]:
score_list = [scores_doc_coref2(doc) for doc in nlp_list]

C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THIN

C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THIN

C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THIN

C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THIN

C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THIN

...

C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THIN

C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THIN

C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THIN

C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THINKPAD\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\THIN

KeyboardInterrupt: 

NameError: name 'score_list' is not defined

In [47]:
final_list=[]
for k in range(len(nlp_list)):
   descr=nlp_list[k]
   list_score=[]
   score_doc=score_list[k]
   for ent in score_doc.keys():
       list_score.append([ent,score_doc[ent]])
    
   final_list.append(list_score)




In [ ]:
ent_load = pd.read_pickle("ent.csv")

In [ ]:
score_doc(doc,doc)

In [ ]:
spacy.__version__

        

In [54]:
def score_sim(doc,k):
    res=0
    score_doc=scores_doc_coref2(doc)
    score_image=score_list[k]
    for i in score_doc.keys():
        if i in score_image:
            res += (score_doc[i]+score_image[i])/2
    return res

def best_image(doc):
    best_score=0
    best_descr=0
    for k in range(len(nlp_list)):
        if score_sim(doc,k) > best_score:
            best_score=score_sim(doc,k)
            best_descr=k
    return nlp_list[best_descr],best_score,best_descr

In [80]:
best_image(nlp_list[34567])

(Members of the South African Police Services fire rubber bullets at striking farmworkers during violent clashes, on January 10, 2013 in De Doorns, a small farming town about 140Km North of Cape Town, South Africa. The farm workers have said that they they will not return to work on the fruit growing region's farms until they receive a daily wage of at least R150($17) per day, which is about double what they currently earn. One of the men wears a red t-shirt of the ruling African National Congress(ANC).  AFP PHOTO / RODGER BOSCH,
 3.1489361700000003,
 13429)

In [72]:
global_ent_list = [[ent.text for ent in NLP.ents if ent_good_type(ent)] for NLP in nlp_list]

def freq_dict(L):
    n = len(L)
    L_unique = list(set(L))
    d = {}
    for a in L_unique:
        for b in L:
            if a == b:
                if a in d.keys():
                    d[a] += 1/n
                else:
                    d[a] = 1/n
    return(d)

global_freq_list = [freq_dict(L) for L in global_ent_list]

def concat_dict(A,B):
    d = {}
    for a in A.keys():
        if a in B.keys():
            if a not in d.keys():
                d[a] = (A[a], B[a])
    return(d)

final_list = [concat_dict(score_list[i], global_freq_list[i]) for i in range(len(score_list))]

In [81]:
nlp_list[34567]

Renee Warnes of South Africa competes during the women's 100m individual medley qualification on December 13, 2012 of the FINA World Short Course Swimming Championships in Istanbul. AFP PHOTO/MIRA

A Lebanese Shiite child beats his forehead with a sword, as part of a mourning ritual, during a procession on the tenth day of the lunar month of Muharram which marks the day of Ashura, in the southern Lebanese city of Nabatieh on September 10, 2019.    The day of Ashura is commemorated by Shiite Muslims worldwide and marks the climax of mourning rituals in the Islamic month of Muharram for the 7th century killing of Imam Hussein, the grandson of Prophet Mohammed, in the Battle of Karbala in 680 AD.

In [2]:
import neuralcoref
help(neuralcoref)

C:\Users\THINKPAD\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: spacy.morphology.Morphology size changed, may indicate binary incompatibility. Expected 104 from C header, got 112 from PyObject
  return f(*args, **kwds)
C:\Users\THINKPAD\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: spacy.vocab.Vocab size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
  return f(*args, **kwds)


Help on package neuralcoref:

NAME
    neuralcoref

PACKAGE CONTENTS
    file_utils
    neuralcoref
    tests (package)
    train (package)

CLASSES
    builtins.object
        neuralcoref.neuralcoref.NeuralCoref
    
    class NeuralCoref(builtins.object)
     |  spaCy v2.0 Coref pipeline component
     |  
     |  Methods defined here:
     |  
     |  __call__(...)
     |      Apply the pipeline component on a Doc object.
     |  
     |  __init__(...)
     |      Create a Coref pipeline component.
     |      vocab (Vocab): The vocabulary object. Must be shared with documents
     |          to be processed. The value is set to the `.vocab` attribute.
     |      model (object): Neural net model. The value is set to the .model attribute. If set to True
     |          (default), a new instance will be created with `NeuralCoref.Model()`
     |          in NeuralCoref.from_disk() or NeuralCoref.from_bytes().
     |      **cfg_inference: Arbitrary configuration parameters. Set to the 